In [2]:
%pylab inline
from astropy.table import Table
from rnn import RNNClassifier
import tensorflow as tf

Populating the interactive namespace from numpy and matplotlib


In [3]:
# Loading dataset
data_table = Table.read('normalized_no_clip.fits')

In [4]:
# Splitting training and testing data
randomize_inds = range(len(data_table))
randomize_inds = permutation(randomize_inds)
randomized_inds_train = randomize_inds[0:15000]
randomized_inds_test  = randomize_inds[15000:]

In [9]:
randomized_inds_test

array([16469, 14620,  8013, ...,  5387, 13706,  1055])

In [10]:
data_table.colnames

['FILENAME',
 'SURVEY',
 'SNID',
 'SNTYPE',
 'FILTERS',
 'RA',
 'DECL',
 'FAKE',
 'MWEBV',
 'REDSHIFT_SPE',
 'NOBS',
 'NVAR',
 'DATA',
 'obs_length']

In [11]:
data_table

FILENAME,SURVEY,SNID,SNTYPE,FILTERS,RA,DECL,FAKE,MWEBV,REDSHIFT_SPE,NOBS,NVAR,"DATA [50,9]",obs_length
bytes16,bytes7,bytes12,bytes5,bytes6,bytes21,bytes20,bytes35,bytes23,bytes25,bytes5,bytes3,float64,float64
DES_SN000018.DAT,DES,18,-9,griz,34.500000 deg,-5.500000 deg,3 (=> BLIND-TEST simulation),0.0227 MW E(B-V),-9.0000 +- 9.0000,118,5,0.0 .. 0.0,0.8341289942655092
DES_SN000050.DAT,DES,50,-9,griz,36.750000 deg,-4.500000 deg,3 (=> BLIND-TEST simulation),0.0283 MW E(B-V),-9.0000 +- 9.0000,16,5,0.0 .. 0.0,0.17683683328855004
DES_SN000051.DAT,DES,51,1,griz,52.500000 deg,-27.500000 deg,3 (=> BLIND-TEST simulation),0.0097 MW E(B-V),0.5177 +- 0.0050,121,5,0.0 .. 0.0,0.7254312291675129
DES_SN000063.DAT,DES,63,-9,griz,34.500000 deg,-5.500000 deg,3 (=> BLIND-TEST simulation),0.0227 MW E(B-V),-9.0000 +- 9.0000,115,5,0.0 .. 0.0,0.731480158026834
DES_SN000064.DAT,DES,64,-9,griz,48.000000 deg,0.000000 deg,3 (=> BLIND-TEST simulation),0.0713 MW E(B-V),-9.0000 +- 9.0000,92,5,0.0 .. 0.0,0.6740610724556567
DES_SN000096.DAT,DES,96,-9,griz,34.500000 deg,-5.500000 deg,3 (=> BLIND-TEST simulation),0.0227 MW E(B-V),-9.0000 +- 9.0000,104,5,0.0 .. 0.0,0.6853870914257367
DES_SN000122.DAT,DES,122,-9,griz,34.500000 deg,-5.500000 deg,3 (=> BLIND-TEST simulation),0.0227 MW E(B-V),-9.0000 +- 9.0000,11,5,0.0 .. 0.0,0.10264311891463129
DES_SN000124.DAT,DES,124,-9,griz,48.000000 deg,0.000000 deg,3 (=> BLIND-TEST simulation),0.0713 MW E(B-V),-9.0000 +- 9.0000,95,5,0.0 .. 0.0,0.6515748369133555
DES_SN000156.DAT,DES,156,-9,griz,36.750000 deg,-4.500000 deg,3 (=> BLIND-TEST simulation),0.0283 MW E(B-V),-9.0000 +- 9.0000,87,5,0.0 .. 0.0,0.5490403471558932


In [12]:
data_table['DATA']

0.0 .. 0.0
0.0 .. 0.0
0.0 .. 0.0
0.0 .. 0.0
0.0 .. 0.0
0.0 .. 0.0
0.0 .. 0.0
0.0 .. 0.0
0.0 .. 0.0
0.0 .. 0.0
0.0 .. 0.0


In [13]:
data_table['label'] = 1.0*(data_table['SNTYPE'].astype('int') == 1)

In [20]:
# Define input function for training 
def input_fn_train():
    def mapping_function(x):
        def extract_batch(inds):
            inds = randomized_inds_train[inds]
            return data_table['label'][inds].astype('float32'), clip(data_table['obs_length'][inds],0,49).astype('int32'),data_table['DATA'][inds].astype('float32') 
        a,b,c =tf.py_func( extract_batch, [x], [tf.float32, tf.int32, tf.float32])
        a.set_shape([None])
        b.set_shape([None])
        c.set_shape([None,50,9])
        return a,b,c

    dataset = tf.data.Dataset.range(len(randomized_inds_train))
    dataset = dataset.repeat().shuffle(10000).batch(512)
    dataset = dataset.map( mapping_function)
    iterator = dataset.make_one_shot_iterator()
    label, length, ts = iterator.get_next()
    return  {'length':length, 'ts':ts}, label

def input_fn_test():
    def mapping_function(x):
        def extract_batch(inds):
            inds = randomized_inds_test[inds]
            return data_table['label'][inds].astype('float32'), clip(data_table['obs_length'][inds],0,49).astype('int32'),data_table['DATA'][inds].astype('float32') 
        a,b,c =tf.py_func( extract_batch, [x], [tf.float32, tf.int32, tf.float32])
        a.set_shape([None])
        b.set_shape([None])
        c.set_shape([None,50,9])
        return a,b,c

    dataset = tf.data.Dataset.range(len(randomized_inds_test))
    dataset = dataset.batch(512)
    dataset = dataset.map( mapping_function)
    iterator = dataset.make_one_shot_iterator()
    label, length, ts = iterator.get_next()
    return  {'length':length, 'ts':ts}, label

In [21]:
model = RNNClassifier(hidden_units=[128, 128], dropout=1.0, model_dir='models/test1')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'models/test1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1267f0240>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [22]:
model.train(input_fn=input_fn_train, steps=200)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from models/test1/model.ckpt-200
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 201 into models/test1/model.ckpt.
INFO:tensorflow:loss = 0.6804806, step = 201
INFO:tensorflow:global_step/sec: 26.1235
INFO:tensorflow:loss = 0.65978014, step = 301 (3.827 sec)
INFO:tensorflow:Saving checkpoints for 400 into models/test1/model.ckpt.
INFO:tensorflow:Loss for final step: 0.646312.


In [23]:
# Apply model to training set
test_prob = [p['prob'] for p in model.predict(input_fn_test)]
table_test = data_table[randomized_inds_test]
table_test['p'] = test_prob

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from models/test1/model.ckpt-400
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [24]:
table_test

FILENAME,SURVEY,SNID,SNTYPE,FILTERS,RA,DECL,FAKE,MWEBV,REDSHIFT_SPE,NOBS,NVAR,"DATA [50,9]",obs_length,label,p
bytes16,bytes7,bytes12,bytes5,bytes6,bytes21,bytes20,bytes35,bytes23,bytes25,bytes5,bytes3,float64,float64,float64,float32
DES_SN179333.DAT,DES,179333,-9,griz,36.750000 deg,-4.500000 deg,3 (=> BLIND-TEST simulation),0.0283 MW E(B-V),-9.0000 +- 9.0000,24,5,0.0 .. 0.0,0.2913376823111847,0.0,0.4720389
DES_SN158542.DAT,DES,158542,-9,griz,34.500000 deg,-5.500000 deg,3 (=> BLIND-TEST simulation),0.0227 MW E(B-V),-9.0000 +- 9.0000,19,5,0.0 .. 0.0,0.21696673089127758,0.0,0.4720389
DES_SN086920.DAT,DES,86920,-9,griz,52.500000 deg,-27.500000 deg,3 (=> BLIND-TEST simulation),0.0097 MW E(B-V),-9.0000 +- 9.0000,125,5,0.0 .. 0.0,0.7938790328574832,0.0,0.4720389
DES_SN085591.DAT,DES,85591,-9,griz,0.500000 deg,-43.000000 deg,3 (=> BLIND-TEST simulation),0.0111 MW E(B-V),-9.0000 +- 9.0000,97,5,0.0 .. 0.0,0.8403551601708602,0.0,0.4720389
DES_SN137668.DAT,DES,137668,-9,griz,34.500000 deg,-5.500000 deg,3 (=> BLIND-TEST simulation),0.0227 MW E(B-V),-9.0000 +- 9.0000,108,5,0.0 .. 0.0,0.6403631644245302,0.0,0.4720389
DES_SN147764.DAT,DES,147764,-9,griz,52.500000 deg,-27.500000 deg,3 (=> BLIND-TEST simulation),0.0097 MW E(B-V),-9.0000 +- 9.0000,141,5,0.0 .. 0.0,0.8287261230254077,0.0,0.4720389
DES_SN063009.DAT,DES,63009,-9,griz,36.750000 deg,-4.500000 deg,3 (=> BLIND-TEST simulation),0.0283 MW E(B-V),-9.0000 +- 9.0000,83,5,0.0 .. 0.0,0.4749381099670114,0.0,0.4720389
DES_SN103059.DAT,DES,103059,-9,griz,34.500000 deg,-5.500000 deg,3 (=> BLIND-TEST simulation),0.0227 MW E(B-V),-9.0000 +- 9.0000,23,5,0.0 .. 0.0,0.2624366091694561,0.0,0.4720389
DES_SN106829.DAT,DES,106829,-9,griz,34.500000 deg,-5.500000 deg,3 (=> BLIND-TEST simulation),0.0227 MW E(B-V),-9.0000 +- 9.0000,19,5,0.0 .. 0.0,0.21696673089127758,0.0,0.4720389
